In [1]:
import warnings
warnings.filterwarnings('ignore')
import nvtabular as nvt
import cudf
import config
import os
# ignore warnings
from merlin.core.dispatch import get_lib
from merlin.schema.tags import Tags

import time
import merlin.models.tf as mm
from tensorflow.keras.callbacks import EarlyStopping

from merlin.schema.tags import Tags
from nvtabular.ops import *

from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

2023-09-09 16:55:08.226250: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-09 16:55:08.251077: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[INFO]: sparse_operation_kit is imported


2023-09-09 16:55:09.260116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 16:55:09.260346: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 16:55:09.260396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.2.0-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Initialize finished, communication tool: horovod


2023-09-09 16:55:09.616715: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 16:55:09.616839: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 16:55:09.616887: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:1013] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/s

In [2]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'train.parquet'),engine="parquet",  part_mem_fraction=0.1)
valid = nvt.Dataset(os.path.join(config.data_processed_dir,'valid.parquet'),engine="parquet", part_mem_fraction=0.1)

In [3]:
# customer_features = ['customer_id', 'age', 'FN', 'Active', 'club_member_status','fashion_news_frequency']
customer_features = ['customer_id', 'age']
article_features = ['article_id', 'product_code', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code', 'index_group_no', 'section_no', 'garment_group_no']

In [4]:
articles = cudf.read_parquet(os.path.join(config.data_raw_dir,'articles.parquet'))[article_features]
customers = cudf.read_parquet(os.path.join(config.data_processed_dir,'customers_enc.parquet'))[customer_features]


In [5]:
articles = nvt.Dataset(articles)
customers = nvt.Dataset(customers)

In [6]:
train = Dataset.merge(train, articles, on="article_id", how="inner")
train = Dataset.merge(train, customers, on="customer_id", how="inner")
train.to_parquet(os.path.join(config.data_processed_dir,'transformer4rec', 'train.parquet'))
valid = Dataset.merge(valid, articles, on="article_id", how="inner")
valid = Dataset.merge(valid, customers, on="customer_id", how="inner")
valid.to_parquet(os.path.join(config.data_processed_dir,'transformer4rec', 'valid.parquet'))
print(len(train.to_ddf()), len(valid.to_ddf()))
del articles, customers, train, valid

8231156 4453834


In [7]:
# ['t_dat', 'customer_id', 'article_id', 'price', 'sales_channel_id','week']

article_id = ["article_id"] >> Categorify() 
customer_id = ["customer_id"] >> Categorify() >> TagAsUserID()
price = ["price"] >> LogOp() >> Normalize() >> TagAsItemFeatures() 
week_split = ["week"] 
time_features = ["t_dat"]
session_time = (
    time_features >> 
    nvt.ops.LambdaOp(lambda col: cudf.to_datetime(col, unit='s')) >> 
    nvt.ops.Rename(name = 'event_time_dt')
)

day = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.day)
        >> nvt.ops.Rename(name = 'day')
        >> Categorify()
        >> TagAsUserFeatures())
week = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.weekday)
        >> nvt.ops.Rename(name = 'week_day')
        >> Categorify()
        >> TagAsUserFeatures())
month = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.month)
        >> nvt.ops.Rename(name = 'month')
        >> Categorify()
        >> TagAsUserFeatures())
year = (session_time
        >> nvt.ops.LambdaOp(lambda col: col.dt.year)
        >> nvt.ops.Rename(name = 'year')
        >> Categorify()
        >> TagAsUserFeatures())

In [8]:
age =  ['age'] >>  LogOp() >> Normalize() >> TagAsUserFeatures() >> AddTags([Tags.USER,Tags.CONTINUOUS])
# cat_customer_columns = (['FN', 'Active', 'club_member_status','fashion_news_frequency'] 
#             >> Categorify() 
#             >> TagAsUserFeatures()
#             )

In [9]:
cat_article_columns = (article_features 
              >> Categorify()  
              >> TagAsItemFeatures())

In [10]:
SESSIONS_MAX_LENGTH =30
SESSIONS_MIN_LENGTH = 2

filtered_articles = (article_id + customer_id + ['t_dat'] + price + day + week + month + year + age  + cat_article_columns + week_split
                        >> Filter(f=lambda df: df["article_id"]!=0)
)
groupby_features = (filtered_articles
                    >> Groupby(
                        groupby_cols=['customer_id', 'week'],
                        aggs={
                            'article_id': ['list', 'count'],
                            'price': ['list','mean', 'std', 'max'],
                            'product_code': ['list'],
                            'product_type_no': ['list'],
                            'graphical_appearance_no': ['list'],
                            'colour_group_code': ['list'],
                            'perceived_colour_value_id': ['list'],
                            'perceived_colour_master_id': ['list'],
                            'department_no': ['list'],
                            'index_code': ['list'],
                            'index_group_no': ['list'],
                            'section_no': ['list'],
                            'garment_group_no': ['list'],
                            
                            'day': ['list'],
                            'week_day': ['list'],
                            'month': ['list'],
                            'year': ['list'],
                        },
                        sort_cols=["t_dat"],
                        name_sep="_"
                    )
                )


In [11]:
list_columns = ['price_list',
 'day_list',
 'week_day_list',
 'month_list',
 'year_list',
 'product_code_list',
 'product_type_no_list',
 'graphical_appearance_no_list',
 'colour_group_code_list',
 'perceived_colour_value_id_list',
 'perceived_colour_master_id_list',
 'department_no_list',
 'index_code_list',
 'index_group_no_list',
 'section_no_list',
 'garment_group_no_list']

groupby_features_articles_id= (groupby_features['article_id_list'] 
                         >> AddTags([Tags.ITEM, Tags.ITEM_ID, Tags.SEQUENCE])
                        )
groupby_features_articles_features= (groupby_features[list_columns]
                         >> AddTags([Tags.ITEM, Tags.SEQUENCE])
                        )
truncated_features = (groupby_features_articles_id + groupby_features_articles_features
                      >> ListSlice(-SESSIONS_MAX_LENGTH) 
                     )
stat_features = (groupby_features['article_id_count','price_max','price_mean','price_std']
               >> LogOp()
               >> Normalize()
               >> Rename(postfix="_norm")
               >> AddTags([Tags.ITEM,Tags.CONTINUOUS])
              )

output = (groupby_features['customer_id',"article_id_count"] + truncated_features + stat_features + week_split
                     >> Filter(f=lambda df: df["article_id_count"] >= SESSIONS_MIN_LENGTH)
                    )

In [12]:
train = nvt.Dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'train.parquet'),engine="parquet",)
valid = nvt.Dataset(os.path.join(config.data_processed_dir,'transformer4rec', 'valid.parquet'),engine="parquet")
print(len(train.to_ddf()), len(valid.to_ddf()))
wf = nvt.Workflow(output)


8231156 4453834


In [13]:
train = wf.fit_transform(train)
valid = wf.transform(valid)
print(len(train.to_ddf()), len(valid.to_ddf()))
# train.regenerate_dataset(os.path.join(config.data_final,'transformer4rec', 'train.parquet'),part_size="500MiB", file_size="500MiB")
# valid.regenerate_dataset(os.path.join(config.data_final,'transformer4rec', 'valid.parquet'),part_size="500MiB", file_size="500MiB")

1576278 868810


In [14]:
train.to_parquet(os.path.join(config.data_final,'transformer4rec', 'train.parquet'))
valid.to_parquet(os.path.join(config.data_final,'transformer4rec', 'valid.parquet'))

In [16]:
train.head()

,customer_id,article_id_count,article_id_list,price_list,day_list,week_day_list,month_list,year_list,product_code_list,product_type_no_list,...,department_no_list,index_code_list,index_group_no_list,section_no_list,garment_group_no_list,article_id_count_norm,price_max_norm,price_mean_norm,price_std_norm,week
0,0,3,"[1731, 16470, 1676]","[0.4938197731971741, -0.5464950203895569, -0.1...","[5, 5, 5]","[5, 5, 5]","[10, 10, 10]","[4, 4, 4]","[113, 9240, 1301]","[4, 56, 5]",...,"[9, 247, 28]","[3, 12, 3]","[3, 7, 3]","[3, 45, 7]","[10, 11, 9]",0.016249,0.244905,0.095169,0.084031,61
1,0,5,"[1126, 3468, 8556, 151, 2055]","[-0.9365506768226624, -0.35728418827056885, 0....","[21, 21, 21, 21, 21]","[3, 3, 3, 3, 3]","[7, 7, 7, 7, 7]","[3, 3, 3, 3, 3]","[17, 1468, 3083, 95, 1417]","[6, 8, 18, 9, 5]",...,"[7, 22, 43, 7, 7]","[3, 3, 4, 3, 3]","[3, 3, 4, 3, 3]","[3, 7, 4, 3, 3]","[3, 3, 14, 3, 3]",0.696710,-0.003786,-0.673464,-0.065536,61
2,1,9,"[22, 31635, 72, 145, 145, 786, 7522, 26482, 137]","[-0.13971909880638123, -0.33306726813316345, 0...","[21, 21, 21, 21, 21, 21, 21, 21, 21]","[9, 9, 9, 9, 9, 9, 9, 9, 9]","[6, 6, 6, 6, 6, 6, 6, 6, 6]","[3, 3, 3, 3, 3, 3, 3, 3, 3]","[82, 11727, 35, 31, 31, 631, 3095, 8905, 4]","[11, 60, 11, 13, 13, 9, 17, 4, 13]",...,"[3, 180, 3, 3, 3, 6, 11, 174, 3]","[5, 8, 5, 5, 5, 4, 3, 10, 5]","[3, 6, 3, 3, 3, 4, 3, 7, 3]","[5, 39, 5, 5, 5, 9, 3, 46, 5]","[7, 11, 7, 7, 7, 5, 8, 22, 7]",1.553989,0.486868,-0.246169,0.141382,61
3,1,9,"[39575, 26482, 4110, 72, 1086, 23096, 7, 137, ...","[-0.9169830083847046, -0.33306726813316345, 1....","[9, 9, 17, 17, 17, 17, 17, 17, 17]","[5, 5, 3, 3, 3, 3, 3, 3, 3]","[6, 6, 6, 6, 6, 6, 6, 6, 6]","[3, 3, 3, 3, 3, 3, 3, 3, 3]","[14163, 8905, 3129, 35, 151, 9386, 8, 4, 45]","[4, 4, 22, 11, 23, 55, 9, 13, 9]",...,"[146, 174, 19, 3, 3, 127, 88, 3, 22]","[10, 10, 4, 5, 5, 10, 4, 5, 3]","[7, 7, 4, 3, 3, 7, 4, 3, 3]","[46, 46, 4, 5, 5, 30, 34, 5, 7]","[3, 22, 9, 7, 7, 22, 5, 7, 3]",1.553989,0.772106,0.176938,0.846301,61
4,1,14,"[10, 10, 988, 988, 988, 988, 105, 66, 1631, 27...","[-0.6242952346801758, -0.6242952346801758, 0.1...","[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]","[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[4, 4, 112, 112, 112, 112, 270, 163, 4, 171, 1...","[13, 13, 11, 11, 11, 11, 11, 11, 13, 13, 13, 1...",...,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]","[7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]",2.234450,-0.078745,-0.090086,-0.347021,61


In [17]:
from transformers4rec.utils.data_utils import save_time_based_splits
save_time_based_splits(train,
                       output_dir=os.path.join(config.data_final,'time'),
                       partition_col='week',
                       timestamp_col='customer_id', 
                      )

Creating time-based splits: 100%|██████████| 8/8 [00:03<00:00,  2.57it/s]
